### Final Project Requirements/notes: https://docs.google.com/document/d/1mwYbYJHkB7kpx4tNflKh54jN9_oOscw3p4k5fsmn3bc/edit

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

### Link with all Data: https://www.sec.gov/dera/data/financial-statement-data-sets.html
- using NUM file only for now (data set of all numeric XBRL facts presented on the primary financial statements)

# 1. Combine frauds data with NonFraud data

In [44]:
frauds = pd.read_csv('fraudsJoinedwithmaster.csv')
qtrs=sorted(frauds.related_qtr.unique().tolist())
len(qtrs) #missing 2q16, will import that df

35

In [45]:
frauds.head()

,adsh,comp_name,related_qtr,filing_type,fraud,tag,version,ddate,qtrs,uom,value
0,0000023082-10-000036,COMPUTER SCIENCES CORP,2q10,10k,1,AccountsPayableCurrent,us-gaap/2009,20090331.0,0.0,USD,6.360000e+08
1,0000023082-10-000036,COMPUTER SCIENCES CORP,2q10,10k,1,AccountsPayableCurrent,us-gaap/2009,20100331.0,0.0,USD,4.090000e+08
2,0000023082-10-000036,COMPUTER SCIENCES CORP,2q10,10k,1,AccruedLiabilitiesCurrent,us-gaap/2009,20090331.0,0.0,USD,1.264000e+09
3,0000023082-10-000036,COMPUTER SCIENCES CORP,2q10,10k,1,AccruedLiabilitiesCurrent,us-gaap/2009,20100331.0,0.0,USD,1.344000e+09
4,0000023082-10-000036,COMPUTER SCIENCES CORP,2q10,10k,1,AccumulatedDepreciationDepletionAndAmortizatio...,us-gaap/2009,20090331.0,0.0,USD,-3.417000e+09


In [46]:
frauds.isnull().sum()

adsh            0
comp_name       0
related_qtr     0
filing_type     0
fraud           0
tag            62
version        62
ddate          62
qtrs           62
uom            62
value          62
dtype: int64

In [47]:
frauds = frauds.dropna()

In [100]:
df = pd.read_csv('numsbyquarter/2018q1/num.txt', sep='\t',encoding ='latin1')
df.head()

,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote
0,0001683168-18-000002,EntityPublicFloat,dei/2014,NaN,20160630,0,USD,3200000.0,NaN
1,0001562762-17-000231,EntityPublicFloat,dei/2014,NaN,20170331,0,USD,28500000.0,NaN
2,0001599916-18-000002,EntityPublicFloat,dei/2014,NaN,20171231,0,USD,253660.0,NaN
3,0001614106-18-000003,EntityPublicFloat,dei/2014,NaN,20161231,0,USD,18247.0,NaN
4,0001144204-18-000327,EntityPublicFloat,dei/2014,NaN,20170331,0,USD,23466799.0,NaN


In [101]:
df.shape

(3030131, 9)

In [102]:
df.isnull().sum()

adsh              0
tag               0
version           0
coreg       2865204
ddate             0
qtrs              0
uom               0
value         50173
footnote    3023631
dtype: int64

In [103]:
df = df.drop(columns=['footnote','coreg'])
df = df.dropna()
df.shape

(2979958, 7)

In [104]:
testfrauds = frauds['adsh']
dftest = df['adsh']

In [105]:
# testfrauds = testfrauds.set_index(['adsh'])
# dftest = dftest.set_index(['adsh'])

In [106]:
dftest.head()

0    0001683168-18-000002
1    0001562762-17-000231
2    0001599916-18-000002
3    0001614106-18-000003
4    0001144204-18-000327
Name: adsh, dtype: object

In [107]:
testfrauds.head()

0    0000023082-10-000036
1    0000023082-10-000036
2    0000023082-10-000036
3    0000023082-10-000036
4    0000023082-10-000036
Name: adsh, dtype: object

In [108]:
result = dftest[dftest.isin(testfrauds)]
result

Series([], Name: adsh, dtype: object)

In [134]:
combined = frauds.append(df)

In [135]:
combined.shape

(3040863, 11)

In [136]:
combined = combined.drop(columns=['comp_name','filing_type','related_qtr'])

In [137]:
combined.isnull().sum()

adsh             0
ddate            0
fraud      2979958
qtrs             0
tag              0
uom              0
value            0
version          0
dtype: int64

In [138]:
combined = combined.fillna(0)

In [139]:
combined.isnull().sum()

adsh       0
ddate      0
fraud      0
qtrs       0
tag        0
uom        0
value      0
version    0
dtype: int64

In [140]:
combined.head()

,adsh,ddate,fraud,qtrs,tag,uom,value,version
0,0000023082-10-000036,20090331.0,1.0,0.0,AccountsPayableCurrent,USD,6.360000e+08,us-gaap/2009
1,0000023082-10-000036,20100331.0,1.0,0.0,AccountsPayableCurrent,USD,4.090000e+08,us-gaap/2009
2,0000023082-10-000036,20090331.0,1.0,0.0,AccruedLiabilitiesCurrent,USD,1.264000e+09,us-gaap/2009
3,0000023082-10-000036,20100331.0,1.0,0.0,AccruedLiabilitiesCurrent,USD,1.344000e+09,us-gaap/2009
4,0000023082-10-000036,20090331.0,1.0,0.0,AccumulatedDepreciationDepletionAndAmortizatio...,USD,-3.417000e+09,us-gaap/2009


In [141]:
combined['tag'] = combined['tag'].str.lower()
combined['2ndindex'] = combined['tag']
combined = combined.set_index(['adsh', '2ndindex'])
combined = combined.loc[combined['uom'] == 'USD']
alltags = combined.tag.unique().tolist()
print(len(alltags))
# for i in allfraudtags:
#     print(i, len(frauds.loc[frauds['tag'] == i ]))

132350


In [142]:
currentassets = "cash|commercialpaper|cashand|accountsrec|accountrec|receivable|inventor|asset|uncollectible|taxben|prepaid|fairval"
longtermassets = "land|building|lease|rental|equipment|machinery|depreciation|investment|intangible|longtermasset|trademark"
liabilities = "payable|short-term|borrowing|debt|liabilit|accrued|lineofcredit|provision|contingen"
shareholders = "othercompre|shareholder|controlling|equity|retained|stock"
income = "revenue|product|services|grossprofit|income|loss|incomeloss|proceeds"
expenses = "costof|costsof|research|expense|services|admin|gainon|cost|paid|compensat|charge"

combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(currentassets)].
                                            index, 'tag', 'CurrentAssets')

combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(longtermassets)].
                                            index, 'tag', 'LongtermAssets')


combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains('goodwill')].
                                            index, 'tag', 'Goodwill')

combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(liabilities)].
                                            index, 'tag', 'Liabilities')


combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(shareholders)].
                                            index, 'tag', 'ShareholdersEquity')


combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(income)].
                                            index, 'tag', 'Income')


combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(expenses)].
                                            index, 'tag', 'Expenses')





In [143]:
remainingtags = combined.tag.unique().tolist()
print(len(remainingtags))

26709


In [144]:
combined.head()

ddate  \
adsh                 2ndindex                                                         
0000023082-10-000036 accountspayablecurrent                              20090331.0   
                     accountspayablecurrent                              20100331.0   
                     accruedliabilitiescurrent                           20090331.0   
                     accruedliabilitiescurrent                           20100331.0   
                     accumulateddepreciationdepletionandamortization...  20090331.0   

                                                                         fraud  \
adsh                 2ndindex                                                    
0000023082-10-000036 accountspayablecurrent                                1.0   
                     accountspayablecurrent                                1.0   
                     accruedliabilitiescurrent                             1.0   
                     accruedliabilitiescurrent                             1.0   
                     accumulateddepreciationdepletionandamortization...    1.0   

                                                                         qtrs  \
adsh                 2ndindex                                                   
0000023082-10-000036 accountspayablecurrent                               0.0   
                     accountspayablecurrent                               0.0   
                     accruedliabilitiescurrent                            0.0   
                     accruedliabilitiescurrent                            0.0   
                     accumulateddepreciationdepletionandamortization...   0.0   

                                                                                    tag  \
adsh                 2ndindex                                                             
0000023082-10-000036 accountspayablecurrent                                 Liabilities   
                     accountspayablecurrent                                 Liabilities   
                     accruedliabilitiescurrent                              Liabilities   
                     accruedliabilitiescurrent                              Liabilities   
                     accumulateddepreciationdepletionandamortization...  LongtermAssets   

                                                                         uom  \
adsh                 2ndindex                                                  
0000023082-10-000036 accountspayablecurrent                              USD   
                     accountspayablecurrent                              USD   
                     accruedliabilitiescurrent                           USD   
                     accruedliabilitiescurrent                           USD   
                     accumulateddepreciationdepletionandamortization...  USD   

                                                                                value  \
adsh                 2ndindex                                                           
0000023082-10-000036 accountspayablecurrent                              6.360000e+08   
                     accountspayablecurrent                              4.090000e+08   
                     accruedliabilitiescurrent                           1.264000e+09   
                     accruedliabilitiescurrent                           1.344000e+09   
                     accumulateddepreciationdepletionandamortization... -3.417000e+09   

                                                                              version  
adsh                 2ndindex                                                          
0000023082-10-000036 accountspayablecurrent                              us-gaap/2009  
                     accountspayablecurrent                              us-gaap/2009  
                     accruedliabilitiescurrent                           us-gaap/2009  
                     accruedliabilitiescurrent                

In [145]:
combined.to_csv('combined_df_bucketed_tags.csv')

In [147]:
bucket_tags = ['CurrentAssets','LongtermAssets','Goodwill','Liabilities','ShareholdersEquity','Income','Expenses']


In [152]:
bucket_df = combined[combined['tag'].isin(['CurrentAssets','LongtermAssets','Goodwill','Liabilities','ShareholdersEquity','Income','Expenses'])]

In [153]:
bucket_df

ddate  \
adsh                 2ndindex                                                         
0000023082-10-000036 accountspayablecurrent                              20090331.0   
                     accountspayablecurrent                              20100331.0   
                     accruedliabilitiescurrent                           20090331.0   
                     accruedliabilitiescurrent                           20100331.0   
                     accumulateddepreciationdepletionandamortization...  20090331.0   
                     accumulateddepreciationdepletionandamortization...  20100331.0   
                     accumulatedothercomprehensiveincomelossnetoftax     20090331.0   
                     accumulatedothercomprehensiveincomelossnetoftax     20100331.0   
                     additionalpaidincapital                             20090331.0   
                     additionalpaidincapital                             20100331.0   
                     allowancefordoubtfulaccountsreceivablecurrent       20090331.0   
                     allowancefordoubtfulaccountsreceivablecurrent       20100331.0   
                     assets                                              20090331.0   
                     assets                                              20100331.0   
                     assetscurrent                                       20090331.0   
                     assetscurrent                                       20100331.0   
                     cashandcashequivalentsatcarryingvalue               20070331.0   
                     cashandcashequivalentsatcarryingvalue               20080331.0   
                     cashandcashequivalentsatcarryingvalue               20090331.0   
                     cashandcashequivalentsatcarryingvalue               20100331.0   
                     cashandcashequivalentsperiodincreasedecrease        20080331.0   
                     cashandcashequivalentsperiodincreasedecrease        20090331.0   
                     cashandcashequivalentsperiodincreasedecrease        20100331.0   
                     commonstockvalue                                    20090331.0   
                     commonstockvalue                                    20100331.0   
                     costofservices                                      20080331.0   
                     costofservices                                      20090331.0   
                     costofservices                                      20100331.0   
                     debtcurrent                                         20090331.0   
                     debtcurrent                                         20100331.0   
...                                                                             ...   
0001493152-18-004278 capitalizedcostsofprovedpropertiesbeingamortiza...  20161231.0   
                     capitalizedcostsofprovedpropertiesbeingamortiza...  20171231.0   
                     capitalizedcostsofprovedpropertiesoilandgasprod...  20161231.0   
                     capitalizedcostsofprovedpropertiesoilandgasprod...  20171231.0   
                     changesinproductionratesandother                    20161231.0   
                     changesinproductionratesandother                    20171231.0   
                     conversionofseriesandseriesbpreferredstocktocom...  20171231.0   
                     costsnotbeingamortized                              20161231.0   
                     costsnotbeingamortized                              20171231.0   
                     costssubjecttoamortization                          20161231.0   
                     costssubjecttoamortization                          20171231.0   
                     debtdiscountfromissuanceofwarrantsasdebtinducement  20171231.0   
                     developmentcostsnotsubjecttoamortization            20171231.0   
                     earningspershareeffectofsharebasedcompensatione...  201

In [154]:
for i in bucket_tags:
    print(i, len(combined.loc[combined['tag'] == i ]))

CurrentAssets 665082
LongtermAssets 275662
Goodwill 19144
Liabilities 326122
ShareholdersEquity 323017
Income 490626
Expenses 259524


In [156]:
len(bucket_df.index.unique())

952672

In [157]:
bucket_df.to_csv('combined_df_filtered_for_7_tags.csv')

In [158]:
df = pd.read_csv('combined_df_filtered_for_7_tags.csv')

In [160]:
len(df.adsh.unique())

6034

In [162]:
fraudsters = df.loc[df.fraud == 1]

In [163]:
len(fraudsters.adsh.unique())

121

In [164]:
nonfrauds = df.loc[df.fraud == 0]
len(nonfrauds.adsh.unique())

5913

In [165]:
121/6034

0.020053032814053694

In [166]:
df.head()

,adsh,2ndindex,ddate,fraud,qtrs,tag,uom,value,version
0,0000023082-10-000036,accountspayablecurrent,20090331.0,1.0,0.0,Liabilities,USD,6.360000e+08,us-gaap/2009
1,0000023082-10-000036,accountspayablecurrent,20100331.0,1.0,0.0,Liabilities,USD,4.090000e+08,us-gaap/2009
2,0000023082-10-000036,accruedliabilitiescurrent,20090331.0,1.0,0.0,Liabilities,USD,1.264000e+09,us-gaap/2009
3,0000023082-10-000036,accruedliabilitiescurrent,20100331.0,1.0,0.0,Liabilities,USD,1.344000e+09,us-gaap/2009
4,0000023082-10-000036,accumulateddepreciationdepletionandamortizatio...,20090331.0,1.0,0.0,LongtermAssets,USD,-3.417000e+09,us-gaap/2009


In [168]:
df.describe(include = 'all')

,adsh,2ndindex,ddate,fraud,qtrs,tag,uom,value,version
count,2359177,2359177,2.359177e+06,2.359177e+06,2.359177e+06,2359177,2359177,2.359177e+06,2359177
unique,6034,105648,NaN,NaN,NaN,7,1,NaN,5773
top,0001628280-18-001324,netincomeloss,NaN,NaN,NaN,CurrentAssets,USD,NaN,us-gaap/2017
freq,4775,35621,NaN,NaN,NaN,665082,2359177,NaN,1728233
mean,NaN,NaN,2.016333e+07,2.237094e-02,2.161360e+00,NaN,NaN,8.305880e+08,NaN
std,NaN,NaN,1.040859e+04,1.478867e-01,1.952828e+00,NaN,NaN,3.108051e+10,NaN
min,NaN,NaN,1.985123e+07,0.000000e+00,0.000000e+00,NaN,NaN,-4.200290e+11,NaN
25%,NaN,NaN,2.016123e+07,0.000000e+00,0.000000e+00,NaN,NaN,1.753000e+04,NaN
50%,NaN,NaN,2.016123e+07,0.000000e+00,3.000000e+00,NaN,NaN,3.900000e+06,NaN
75%,NaN,NaN,2.017123e+07,0.000000e+00,4.000000e+00,NaN,NaN,5.850000e+07,NaN


In [171]:
df_for_pivot = df.drop(columns= ['2ndindex', 'qtrs', 'uom','version'])
df_for_pivot.head()

,adsh,ddate,fraud,tag,value
0,0000023082-10-000036,20090331.0,1.0,Liabilities,6.360000e+08
1,0000023082-10-000036,20100331.0,1.0,Liabilities,4.090000e+08
2,0000023082-10-000036,20090331.0,1.0,Liabilities,1.264000e+09
3,0000023082-10-000036,20100331.0,1.0,Liabilities,1.344000e+09
4,0000023082-10-000036,20090331.0,1.0,LongtermAssets,-3.417000e+09


In [169]:
df.value.sum()

1959504099034806.2

In [176]:
bucket_tag_sum = 0
for i in bucket_tags:
    combined_pivot[i].sum() += bucket_tag_sum

bucket_tag_sum
# for i in bucket_tags:
#     print(i, len(combined.loc[combined['tag'] == i ]))

SyntaxError: can't assign to function call (<ipython-input-176-1640f319b351>, line 3)

In [178]:
combined_pivot["CurrentAssets"].sum()

671622798610447.0

In [183]:
combined_pivot = pd.pivot_table(df_for_pivot,index=['fraud',"adsh","ddate",], columns= ['tag'], values= ['value'], aggfunc='sum')
# explore2.head()

In [184]:
combined_pivot

value                \
tag                                   CurrentAssets      Expenses   
fraud adsh                 ddate                                    
0.0   0000002178-18-000009 20141231.0  9.857100e+07           NaN   
                           20151231.0  1.026290e+08  3.847024e+09   
                           20160331.0           NaN           NaN   
                           20160630.0           NaN           NaN   
                           20160930.0           NaN           NaN   
                           20161231.0  7.170870e+08  2.229644e+09   
                           20170331.0           NaN           NaN   
                           20170430.0 -3.630000e+05  1.909000e+06   
                           20170531.0           NaN           NaN   
                           20170630.0           NaN           NaN   
                           20170831.0           NaN  4.000000e+05   
                           20170930.0           NaN           NaN   
                           20171231.0  8.748190e+08  2.633350e+09   
      0000002488-18-000042 20130930.0           NaN           NaN   
                           20141231.0  8.330000e+08           NaN   
                           20151231.0  1.140000e+09  4.681000e+09   
                           20160331.0           NaN  8.420000e+08   
                           20160630.0           NaN  9.850000e+08   
                           20160831.0           NaN           NaN   
                           20160930.0           NaN  1.548000e+09   
                           20161231.0  2.323400e+10  6.401000e+09   
                           20170331.0           NaN  9.510000e+08   
                           20170630.0           NaN  1.129000e+09   
                           20170930.0           NaN  1.416000e+09   
                           20171231.0  1.981400e+10  6.814000e+09   
      0000002969-18-000014 20160930.0  1.330800e+09           NaN   
                           20161231.0  1.917000e+08  1.501000e+08   
                           20170930.0  4.787950e+10  2.362000e+08   
                           20171231.0  4.587050e+10  6.261000e+08   
      0000003499-18-000006 20141231.0  6.248340e+08           NaN   
...                                             ...           ...   
1.0   0001193125-16-701058 20160630.0  3.665264e+10  2.874215e+09   
      0001193125-16-759676 20140930.0           NaN  1.290000e+07   
                           20141231.0  8.035220e+08           NaN   
                           20150930.0  2.915845e+09  3.957418e+09   
                           20151231.0  3.886842e+10           NaN   
                           20160430.0           NaN           NaN   
                           20160531.0  1.500000e+01           NaN   
                           20160731.0  1.500000e+01           NaN   
                           20160831.0           NaN           NaN   
                           20160930.0  3.965233e+10  3.844138e+09   
      0001354488-11-003677 20100430.0  2.994634e+06           NaN   
                           20100731.0 -3.011267e+06  9.661529e+06   
                           20110430.0  5.651878e+08  4.907666e+07   
                           20110731.0  6.084990e+08  7.047002e+07   
      0001354488-11-004960 20100430.0  2.994634e+06           NaN   
                           20101031.0 -2.972908e+06  3.286408e+07   
                           20110430.0  5.651878e+08  4.907666e+07   
                           20111031.0  6.022836e+08  1.210155e+08   
      0001354488-12-001090 20100430.0  2.994634e+06           NaN   
                           20110131.0  6.121031e+06  3.626925e+07   
                           20110430.0  5.651878e+08  4.907666e+07   
                           20120131.0  6.085809e+08  1.344342e+08   
      0001558370-15-002707 20131231.0  2.416000e+06           NaN   
                           20140331.0           NaN  2.212000e+06   
                           20140630.0       

In [189]:
combined_pivot.to_excel('first_model_data.xls')

In [186]:
first_model= pd.read_csv('first_model_data.csv') #now tags_bucketed_for_first_model.csv

In [187]:
first_model

,Unnamed: 0,Unnamed: 1,Unnamed: 2,value,value.1,value.2,value.3,value.4,value.5,value.6
0,tag,NaN,NaN,CurrentAssets,Expenses,Goodwill,Income,Liabilities,LongtermAssets,ShareholdersEquity
1,fraud,adsh,ddate,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0000002178-18-000009,20141231.0,98571000.0,NaN,NaN,NaN,NaN,NaN,157497000.0
3,0.0,0000002178-18-000009,20151231.0,102629000.0,3847024000.0,NaN,4323836999.7,-87427000.0,95048000.0,151235000.88
4,0.0,0000002178-18-000009,20160331.0,NaN,NaN,NaN,254300000.37,NaN,-0.03,1554000.0
5,0.0,0000002178-18-000009,20160630.0,NaN,NaN,NaN,302168000.84,NaN,-0.03,3540000.0
6,0.0,0000002178-18-000009,20160930.0,NaN,NaN,NaN,252901999.77,NaN,-0.28,-983000.0
7,0.0,0000002178-18-000009,20161231.0,717087000.0,2229644000.0,NaN,2825005000.9,549317000.0,601421999.66,294706001.98
8,0.0,0000002178-18-000009,20170331.0,NaN,NaN,NaN,300642999.8,NaN,0.0,-860000.0
9,0.0,0000002178-18-000009,20170430.0,-363000.0,1909000.0,NaN,-189000.0,NaN,6920000.0,NaN
